This is a quick tutorial for a Naive Conversational RAG using Langchain

#### Installing necessary packages

In [1]:
!pip install -qU \
    langchain \
    langchain-openai \
    langchain-pinecone \
    openai \
    pinecone-client \
    PyMuPDF \
    tiktoken


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.0.20 requires langchain-core<0.2,>=0.1.21, but you have langchain-core 0.3.28 which is incompatible.
langchain-community 0.0.20 requires langsmith<0.1,>=0.0.83, but you have langsmith 0.1.147 which is incompatible.


Using GPT 3.5 Turbo as the LLM

In [2]:
import os
from langchain_openai.chat_models.base import ChatOpenAI
# from langchain_openai import ChatOpenAI

openai_key = "YOUR_OPENAI_API_KEY"
os.environ["OPENAI_API_KEY"] = openai_key
chat = ChatOpenAI(
    openai_api_key=openai_key,
    model='gpt-3.5-turbo')


Using LangChain SystemMessage to prime the behavior of this conversational RAG system

In [3]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant.")
]


### Processing the PDF into Documents

* Step 1: Extract text <br>
Using pymupdf to extract text from PDF - In this exercise, I'm not using tables or figures.

* Step 2: Pre-process text - Tokenize all the text, then create chunks <br>
** Using tiktoken - an OpenSource tokenizer from OpenAI <br>
** Using Encoding='cl100k_base' which is suitable for gpt-3.5-turbo <br>
** For chunking: Using token-level chunking for simplicity. Small chunks may fragment sentences, while large chunks might include irrelevant context. For this exercise, chunk_size=512 (256-512 chunk size is [suggested](https://www.pinecone.io/learn/chunking-strategies/#Embedding-short-and-long-content) for text-embedding-ada-002).<br>

* Step 3: Generate embeddings <br>
Using the OpenAI model "text-embedding-ada-002". This generates 1536 dimensional vectors



In [4]:
import openai
import pinecone
import pymupdf
import tiktoken
from langchain_openai import OpenAIEmbeddings
embed_model = OpenAIEmbeddings(model="text-embedding-ada-002")


In [25]:
# Step 1: Extract text

class PDFLoader:
    def __init__(self, pdf_path):
        self.pdf_path = pdf_path

    def extract_text(self):
        doc = pymupdf.open(self.pdf_path)
        text = ""
        for page in doc:
            text += page.get_text()
        return text

loader = PDFLoader('attention.pdf')
text = loader.extract_text()
# print(text)


In [6]:
# Step 2, 3: Pre-process text, Generate embeddings

class Preprocess:
    def __init__(self):
        openai_api_key = openai_key
        if not openai_api_key:
            raise ValueError("OPENAI_API_KEY environment variable not set")

    def chunkify(self, text, chunk_size, encoding_name="cl100k_base"):
        encoding = tiktoken.get_encoding(encoding_name)
        tokens = encoding.encode(text)
        return [encoding.decode(tokens[i:i + chunk_size]) for i in range(0, len(tokens), chunk_size)]

    def embed(self, chunks):
        embeddings = []
        for chunk in chunks:
            response = openai.embeddings.create(
                input=chunk,
                model=embed_model.model
            )
            embeddings.append(response.data[0].embedding)
        return embeddings

    def chunkify_and_embed(self, text, chunk_size=512):
        chunks = self.chunkify(text, chunk_size)
        embeddings = self.embed(chunks)
        return chunks, embeddings

preprocessor = Preprocess()
chunks, embeddings = preprocessor.chunkify_and_embed(text, chunk_size=512)


In [7]:
print (type(chunks), len(chunks))
type(embeddings[0])


<class 'list'> 20


list

In [8]:
len(embeddings)

20

### Creating Vector Store
Using Pinecone free account.

Step 1: Creating an index. Dimension = 1536 same as embeddings <br>
Step 2: Upserting each chunk's embedding+metadata into the index. We are not making use of metadata for any filtering, but it's still good practice to maintain unique metadata in the vectorstore

In [16]:
### Index and VectorStore creation

import time
import os
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
api_key = "YOUR_PINECONE_API_KEY"
os.environ["PINECONE_API_KEY"] = api_key
pc = Pinecone(api_key=api_key)

index_name = "attention-vector-store"

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)
vector_store = PineconeVectorStore(index=index, embedding=OpenAIEmbeddings())

# Adding Documents
from langchain_core.documents import Document

documents = [
    Document(page_content=text, metadata={"source": f"document_{i}"})
    for i, text in enumerate(chunks)
]

vector_store.add_documents(documents=documents, ids=[str(i) for i in range(len(documents))])
index.describe_index_stats()


{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 20}},
 'total_vector_count': 20}

## RAG

The top k Nearest Neighbor matches of the query from the vector store will be passed to the chatbot as context for the query. The query + context will be called the augmented prompt.

### Conversational RAG
For the chatbot to have conversational memory, we will append the augmented-prompt to the messages. This allows the bot to access conversational history.



In [17]:
def augment_prompt(query: str):
    # Fetch top 3 matches
    results = vector_store.similarity_search(query, k=3)
    source_knowledge = "\n".join([x.page_content for x in results])
    augmented_prompt = f"""Use the following pieces of context to answer the query at the end.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.

    Contexts:
    {source_knowledge}

    Query: {query}"""
    return augmented_prompt

In [19]:
prompt = HumanMessage(
    content=augment_prompt(
        "What is a Transformer?"
    )
)
messages.append(prompt)
res = chat(messages)
print(res.content)


A Transformer is a model architecture that is based entirely on attention mechanisms, replacing the recurrent layers commonly used in encoder-decoder architectures. The Transformer model allows for more efficient parallelization and can establish a new state of the art in translation quality with significantly faster training times compared to models based on recurrent or convolutional layers.


<b> As we can see in the following code cell, the Chatbot is able to understand that 'it' in the query refers to a transformer: "How does it compare with an RNN? List benefits or shortcomings." </b>

In [20]:
# Create a new augmented prompt and append to conversational history
prompt = HumanMessage(
    content=augment_prompt(
        "How does it compare with an RNN? List benefits or shortcomings."
    )
)
messages.append(prompt)
res = chat(messages)
print(res.content)

The Transformer model differs from recurrent neural networks (RNNs) in several ways, offering both benefits and shortcomings when compared to RNN-based architectures:

Benefits of Transformer over RNN:
1. **Parallelization**: The Transformer model allows for significantly more parallelization compared to RNNs. This feature is crucial for training efficiency, especially with longer sequence lengths.
2. **Training Speed**: Transformers can be trained faster than architectures based on RNNs due to their parallel nature, making them more time-efficient.
3. **Global Dependencies**: Transformers utilize an attention mechanism to draw global dependencies between input and output, enabling them to model dependencies without regard to their distance in the sequences.
4. **Interpretable Models**: Self-attention in Transformers could potentially yield more interpretable models, as individual attention heads in the model are observed to perform different tasks related to the syntactic and semantic

In [21]:
prompt = HumanMessage(
    content=augment_prompt(
        "How was its evaluation performed?"
    )
)
messages.append(prompt)
res = chat(messages)
print(res.content)


The evaluation of the Transformer model was performed on the English constituency parsing task. The Transformer was trained on the Wall Street Journal (WSJ) portion of the Penn Treebank, which consisted of about 40K training sentences. Additionally, it was trained in a semi-supervised setting using larger high-confidence and Berkeley Parser corpora with approximately 17M sentences. 

The evaluation involved training a 4-layer Transformer with a specific model size on the WSJ data and the semi-supervised data. Different vocabularies were used for these settings. The evaluation included selecting optimal dropout rates, attention mechanisms, residual connections, learning rates, and beam sizes on the Section 22 development set. Other parameters were kept consistent with the English-to-German base translation model during inference. 

The results of the evaluation were presented in a table, showing the performance of the Transformer model on English constituency parsing compared to other m

Deleting index to save resources:

In [27]:
pc.delete_index(index_name)


### Some general Areas for Experimentation while developing a RAG Chatbot:
1. Pre-processing -
* * Choice of PDF Parsers
** Choice of Tokenizer and encoding strategy
** Chunking strategies - Chunking method (Semantic / sentence / token based), chunk overlap etc
** Embedding models
** Including Tables, Images

2. Choice of LLM
3. Deciding 'k' for Nearest Neighbor Search - Needs to be based on the token limit of LLM
4. Choice of Vector databases
5. Choice of different Retrieval methods: Self Query Retrieval (Metadata-based Filtering), Parent Document Retrieval etc
6. Few-shot learning

Resources:
https://towardsdatascience.com/advanced-retriever-techniques-to-improve-your-rags-1fac2b86dd61
https://github.com/pinecone-io/examples/blob/master/learn/generation/langchain/rag-chatbot.ipynb
